In [5]:
import copy
import math

import networkx as nx

from solution import *

In [6]:
def build_subgraphs(network):

    # Build full networkx graph
    G = nx.DiGraph()
    for src in network:
        if hasattr(network[src], 'dst_modules'):
            for dst in network[src].dst_modules:
                G.add_edge(src, dst)

    # Compute subgraphs
    last_conjunction_inputs = list(network['gh'].input_signals.keys())
    subgraphs = []
    for lci in last_conjunction_inputs:
        asp = nx.all_simple_paths(G, 'broadcaster', lci)
        subgraph_nodes = set(n for path in asp for n in path)
        subgraph_nodes.add('gh')
        subgraph = {name: module for name, module in network.items() if name in subgraph_nodes}
        subgraph = copy.deepcopy(subgraph)
        for module in subgraph.values():
            if hasattr(module, 'dst_modules'):
                module.dst_modules = tuple([dst for dst in module.dst_modules if dst in subgraph_nodes])
            if hasattr(module, 'input_signals'):
                module.input_signals = {k: v for k, v in module.input_signals.items() if k in subgraph_nodes}

        subgraphs.append(subgraph)

    return subgraphs

In [7]:
def get_loop_size_subgraph(subgraph, n_press_button, verbose=0):
    indxs = []
    for n in range(n_press_button):
        queue = deque([('button', 'low', subgraph['broadcaster'])])
        while queue:
            module_name_previous, input_signal, module = queue.popleft()
            if module.name == 'gh' and input_signal == 'high':
                indxs.append(n)
            dst_modules_input_signals = module.send_pulse(input_signal=input_signal, 
                                                          module_name_previous=module_name_previous,
                                                          network=subgraph)
            queue.extend(dst_modules_input_signals)

    loop_size = indxs[1] - indxs[0]  # Cycle starts at 0
    return loop_size

In [10]:
network = parse_input('puzzle.txt')
subgraphs = build_subgraphs(network)

subgraph_loop_sizes = []
for subgraph in subgraphs:
    subgraph_loop_sizes.append(get_loop_size_subgraph(subgraph, n_press_button=10000, verbose=0))

math.lcm(*subgraph_loop_sizes)

226732077152351

In [165]:
85858 - 82125

3733